In [ ]:
#Sejam bem-vindos ao Notebook destinado a explicação prática de uma validação cruzada. Antes de começar vale informar que o conjunto de dados
#utilizado trata-se de informações sobre flores. Vamos nessa? 

In [11]:
import pandas as pd

#Importando os dados e os transformando num DataFrame
url = "https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data"
names = ['sepal-length', 'sepal-width', 'petal-length', 'petal-width', 'class']
df = pd.read_csv(url, names=names)

#Para pré-visualizarmos nossos dados
df.head()

,sepal-length,sepal-width,petal-length,petal-width,class
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [9]:
#Quantidade de linhas e colunas

df.shape

(150, 5)

In [24]:
# Separando as variáveis independentes e a classe
X = df.iloc[:, :-1]  # todas as colunas menos a última
y = df.iloc[:, -1]   # apenas a última coluna

In [13]:
#Proporção da classe

df['class'].value_counts(1)

class
Iris-setosa        0.333333
Iris-versicolor    0.333333
Iris-virginica     0.333333
Name: proportion, dtype: float64

In [26]:
#Importando a função que irá realizar a divisão dos dados em treino e teste

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 42, stratify = y)

print(f"Quantidade de linhas e colunas para os dados de treino: {X_train.shape}")
print(f"Quantidade de linhas e colunas para os dados de teste: {X_test.shape}")

Quantidade de linhas e colunas para os dados de treino: (120, 4)
Quantidade de linhas e colunas para os dados de teste: (30, 4)


In [28]:
# Transformando o array y_train em uma série para analisar a 
# proporção das classes no conjunto de treino
pd.Series(y_train).value_counts(1)

class
Iris-setosa        0.333333
Iris-virginica     0.333333
Iris-versicolor    0.333333
Name: proportion, dtype: float64

In [32]:
# Transformando o array y_test em uma série para analisar a 
# proporção das classes no conjunto de teste
pd.Series(y_test).value_counts(1)

class
Iris-setosa        0.333333
Iris-virginica     0.333333
Iris-versicolor    0.333333
Name: proportion, dtype: float64

In [83]:
# A regressão logística exige que as variáveis estejam escalonadas
# (na mesma ordem de grandeza). 
# Para realizar o escalonamento é importada a função StandardScaler.
from sklearn.preprocessing import StandardScaler

# Instanciando o StandardScaler
scaler = StandardScaler()

# Calcula os parâmetros da escala e aplica aos dados de treino, transformando-os
X_train = scaler.fit_transform(X_train)
# Com os parâmetros já calculados, os dados de teste são escalonados 
X_test = scaler.transform(X_test) 

In [85]:
# importando a função da regressão logística
from sklearn.linear_model import LogisticRegression

# instanciando e treinando o modelo
modelo = LogisticRegression()
modelo.fit(X_train, y_train)

# Obtendo as probabilidades das classes previstas
y_pred_proba = modelo.predict_proba(X_test)

# Obtendo as previsões do modelo
y_pred = modelo.predict(X_test)

In [87]:
# importando as funções para calcular a precisão e a revocação
from sklearn.metrics import precision_score, recall_score

# importando a função para calcular a acurácia do modelo
from sklearn.metrics import accuracy_score

precisao = precision_score(y_test, y_pred, average = "macro")
revocacao = recall_score(y_test, y_pred, average = "macro")
acuracia = accuracy_score(y_test, y_pred)

print(f"A precisão foi de: {precisao:.3f}")
print(f"A revocação foi de: {revocacao:.3f}")
print(f"A acurácia foi de: {acuracia:.3f}")

A precisão foi de: 1.000
A revocação foi de: 1.000
A acurácia foi de: 1.000


In [ ]:
#Após isso vamos para a aplicação da nossa validação cruzada?

In [89]:
# importando a biblioteca Numpy, para manipulações algébricas
import numpy as np

# importando as funções Stratified K-Fold
from sklearn.model_selection import KFold

# Para realizar o escalonamento é importada a função StandardScaler.
from sklearn.preprocessing import StandardScaler

# importando a função da regressão logística
from sklearn.linear_model import LogisticRegression

# importando as funções para calcular a precisão e a revocação
from sklearn.metrics import precision_score, recall_score

# importando a função para calcular a acurácia do modelo
from sklearn.metrics import accuracy_score

In [91]:
# Definindo o número de folds
k = 5

# Inicializando a função StratifiedKFold
folds = KFold(n_splits=k, shuffle=True, random_state=42)

# Criando listas para armazenar os valores de precisão, revocação e 
# acurácia em cada fold
precisoes = list()
revocacoes = list()
acuracias = list()

# Instanciando o StandardScaler
scaler = StandardScaler()

# Transformando X e y em respectivamente, um dataframe e uma série do pandas. 
# Isto é feito para se ter acesso aos índices de cada instância.

X = df.drop(columns = ["class"], axis = 1)
y = df['class']

In [93]:
# Será aplicado o método "split" no objeto folds, que retornará uma lista 
# com os índices das instâncias que pertencem ao conjunto de treino e 
# outra com os índices das instâncias que pertencem ao conjunto de teste

for k, (train_index, test_index) in enumerate(folds.split(X, y)):
    print("=-"*6 + f"Fold: {k+1}" + "-="*6)

    # Dividindo os dados em treino e teste para cada um dos folds
    X_train, X_test = X.iloc[train_index], X.iloc[test_index]
    y_train, y_test = y[train_index], y[test_index]
    # train_index e test_index: São os índices das instâncias do conjunto 
    # de treino e teste, respectivamente, selecionados em cada um dos folds
    
    # Escalonando os dados. Todas as colunas serão passadas para uma 
    # distribuição normal, garantindo que as características estejam
    # em uma mesma escala numérica
    X_train = scaler.fit_transform(X_train) 
    X_test = scaler.transform(X_test) 
    
    # instanciando e treinando o modelo
    modelo = LogisticRegression()
    modelo.fit(X_train, y_train)

    # Obtendo as probabilidades das classes previstas
    y_pred_proba = modelo.predict_proba(X_test)

    # Obtendo as previsões do modelo
    y_pred = modelo.predict(X_test)
    
    # Calculando a precisão, revocação e acurácia para o fold em questão
    precisao = precision_score(y_test, y_pred, average = "weighted")
    revocacao = recall_score(y_test, y_pred, average = "weighted")
    acuracia = accuracy_score(y_test, y_pred)
    
    # Armazenando as precisões, revocações e acurácias nas listas criadas
    precisoes.append(precisao)
    revocacoes.append(revocacao)
    acuracias.append(acuracia)
    
    # Exibindo as métricas para cada um dos folds
    print(f"Precisão: {precisao:.3f}")
    print(f"Revocação: {revocacao:.3f}")
    print(f"Acurácia: {acuracia:.3f}")

=-=-=-=-=-=-Fold: 1-=-=-=-=-=-=
Precisão: 1.000
Revocação: 1.000
Acurácia: 1.000
=-=-=-=-=-=-Fold: 2-=-=-=-=-=-=
Precisão: 0.970
Revocação: 0.967
Acurácia: 0.967
=-=-=-=-=-=-Fold: 3-=-=-=-=-=-=
Precisão: 0.947
Revocação: 0.933
Acurácia: 0.933
=-=-=-=-=-=-Fold: 4-=-=-=-=-=-=
Precisão: 0.903
Revocação: 0.900
Acurácia: 0.900
=-=-=-=-=-=-Fold: 5-=-=-=-=-=-=
Precisão: 0.969
Revocação: 0.967
Acurácia: 0.967


In [95]:
# Transformando as listas precisões, revocações, acurácias em arrays, 
# para fazer operações matemáticas
precisoes = np.array(precisoes)
revocacoes = np.array(revocacoes)
acuracias = np.array(acuracias)
   
# Calculando a média de todas as precisões, revocações e acurácias 
media_precisao = np.mean(precisoes)
media_revocacao = np.mean(revocacoes)
media_acuracia = np.mean(acuracias)

# Calculando o desvio padrão de todas as precisões, revocações e acurácias
std_precisao = np.std(precisoes)
std_revocacao = np.std(revocacoes)
std_acuracia = np.std(acuracias)

# Exibindo a média das precisões e revocações
print(f"Média da precisão: {media_precisao:.3f} +/- {std_precisao:.3f}")
print(f"Média da revocação: {media_revocacao:.3f} +/- {std_revocacao:.3f}")
print(f"Média da acurácia: {media_acuracia:.3f} +/- {std_acuracia:.3f}")

Média da precisão: 0.958 +/- 0.032
Média da revocação: 0.953 +/- 0.034
Média da acurácia: 0.953 +/- 0.034


In [ ]:
#Espero que você tenha gostado de todo esse processo! Vamos lá continuar evoluindo!!